In [1]:
# (ignore this) imports and utility functions

import re
import json
import toolkit

from textwrap import dedent
from IPython.display import clear_output, display, Markdown


def render_markdown(text):
    if text is None:
        text = ""

    pattern = r"```thinking\n(.*?)\n```"

    def replacement(match):
        content = match.group(1)
        return f"<blockquote>\n{content}\n</blockquote>\n\n"

    text = re.sub(pattern, replacement, text, flags=re.DOTALL)

    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    display(Markdown(text))


def render_json(data):
    return render_markdown(f"```json\n{json.dumps(data, indent=2)}\n```\n")


def render_live(stream):
    response = ""
    render_markdown("> Loading...")

    for chunk in stream:
        response += chunk
        clear_output(wait=True)
        render_markdown(response)

    return response

In [2]:
model = "gemma3:27b"

In [3]:
with open("schemas/function.schema.json") as file:
    schema = json.load(file)

In [4]:
await toolkit.code.create_sandbox()

In [5]:
preface = dedent("""
    When generating code to access/interact with external systems for the user, please generate the
    code in Python only. Follow the below guidelines as well:

    - Do not generate examples that use the function.
    - Do not print the result, return it from the function.
    - Do not handle errors by printing or ignoring them, raise a custom exception with a user-friendly
      name and message instead.
    - Python is sensitive to indentation, so do not generate blank lines with only tabs or whitespaces.
""")

task = dedent("""
    I'd like to be able to ask you for the weather. Could you please write code to use the wttr.in API
    to fetch the weather for me when I need it? Just make a request to this URL:

            "http://wttr.in/<place>?format=(%l) %C and %t (feels %f) with winds at %w"
""")

In [6]:
messages = [{"role": "user", "content": preface}, {"role": "user", "content": task}]

In [7]:
streaming_response = toolkit.model.get_response(model, messages)
function_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": function_response})

```python
import requests

class WeatherError(Exception):
    """Custom exception for weather-related errors."""
    pass

def get_weather(place):
    """
    Fetches the weather for a given place using the wttr.in API.

    Args:
        place (str): The place to fetch the weather for.

    Returns:
        str: The weather information.

    Raises:
        WeatherError: If the request fails or the response is invalid.
    """
    url = f"http://wttr.in/{place}?format=(%l) %C and %t (feels %f) with winds at %w"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.text
    except requests.exceptions.RequestException as e:
        raise WeatherError(f"Failed to fetch weather for {place}: {e}") from e
    except Exception as e:
        raise WeatherError(f"An unexpected error occurred: {e}") from e
```

In [8]:
task = dedent(f"""
    Generate a function specification for the function you just generated. Make sure it is produced in a code block
    with the language set to `function_spec` instead of `json`. Make sure to follow the below schema:

    {json.dumps(schema, indent=2)}
""")

In [9]:
messages.append({"role": "user", "content": task})

In [10]:
streaming_response = toolkit.model.get_response(model, messages)
specification_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": specification_response})

```json
{
  "name": "get_weather",
  "description": "Fetches the current weather conditions for a given location using the wttr.in API. The API returns a formatted string with location, temperature, wind speed, and a short description.",
  "parameters": {
    "type": "object",
    "properties": {
      "place": {
        "type": "string",
        "description": "The location for which to retrieve the weather.  This can be a city name, city and country, or an airport code."
      }
    },
    "required": [
      "place"
    ]
  },
  "responses": [
    {
      "type": "string",
      "description": "A string containing the weather information for the specified location. The format of the string is determined by the wttr.in API."
    }
  ],
  "errors": [
    {
      "name": "WeatherError",
      "description": "Raised when there is an issue fetching the weather, such as a network error, an invalid location, or a problem with the wttr.in API."
    }
  ],
  "examples": []
}
```

In [11]:
function = toolkit.functions.parse_funcs(function_response)[0]
specification = toolkit.functions.parse_specs(specification_response)[0]

function = await toolkit.code.define_function(specification["name"], function)
toolkit.functions.register(function, specification)

In [12]:
prompt = toolkit.prompts.function_calling(model)
task = "What's the weather in Pune and Singapore?"

In [13]:
messages.append({"role": "user", "content": prompt})
messages.append({"role": "user", "content": task})

In [14]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
Okay, the user wants to know the weather in Pune and Singapore. I have access to the `get_weather` function which can fetch the weather for a given place. I need to call this function twice, once for Pune and once for Singapore. The calls are independent, so I can execute them in parallel.

Guidelines followed:
- Identified the need for a function call.
- Determined that the function calls are independent and can be done in parallel.
- Will call the function twice to fetch data for two locations.

Guidelines not followed: None.
</blockquote>



```json
{
	"id": "1",
	"function": "get_weather",
	"parameters": {
		"place": "Pune"
	}
}
```

```json
{
	"id": "2",
	"function": "get_weather",
	"parameters": {
		"place": "Singapore"
	}
}
```

In [15]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
render_markdown(outputs)

```json
{
    "id": "1",
    "result": "(Pune) Patchy rain nearby and +23\u00b0C (feels +25\u00b0C) with winds at \u219215km/h"
}
```

```json
{
    "id": "2",
    "result": "(Singapore) Partly cloudy and +30\u00b0C (feels +36\u00b0C) with winds at \u219710km/h"
}
```


In [16]:
messages.append({"role": "user", "content": outputs})

In [17]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

The weather in Pune is patchy rain with a temperature of 23°C (feels like 25°C) and winds at 15km/h. The weather in Singapore is partly cloudy with a temperature of 30°C (feels like 36°C) and winds at 10km/h.